In [4]:
import os

import numpy as np
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from matplotlib import ticker
import seaborn as sns
import pandas as pd
import numpy_indexed as npi
from scipy.io import loadmat

from src.ipm import NeuronLoader, grablabels
from src.metrics import sepind

from imblearn.under_sampling import RandomUnderSampler as rus
from scipy.ndimage import gaussian_filter1d as smooth

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split as tts


In [5]:
fontsize = 28
titlefontsize = 36
make_smooth = True
sns.set_palette("Set2")

# Visualization

In [6]:
def myfmt(x, pos):
    return '{0:.1f}'.format(x)

In [7]:
titlefontsize = 32
fontsize = 24

monkey = "both"
selectivity = "fast"
out_path = f"G:/Codes/Processing/out/svm-tt/{monkey}-{selectivity}"
metric = "dpr"

for region in ["it", "pfc"]:
    fig, axs = plt.subplots(1, 2, figsize=np.array([30, 13]), dpi=30)#/2.54)
    time = np.load(os.path.join(out_path, f"{region.lower()}-super-ordinate-tim.npy"))
    for ax, ordination in zip(axs, ["super-ordinate", "mid-level"]):
        m = np.load(os.path.join(out_path, f"{region.lower()}-{ordination}-{metric}.npy"))
        h = ax.contourf(time, time, m.mean(2), 400, cmap='inferno')
        cbar = fig.colorbar(h, ax=ax, shrink=.6, format=ticker.FuncFormatter(myfmt))
        cbar.ax.tick_params(labelsize=fontsize)
        ax.set_title(ordination, fontsize=fontsize)

        ax.axis("square")
        ax.set_xlabel('train time (ms)', fontsize=fontsize)
        ax.set_ylabel('test time (ms)', fontsize=fontsize)
        ax.tick_params(axis='both', which='major', labelsize=fontsize)

    fig.suptitle(f"{region.upper()}: d-prime of Distance of Observations from Hyper-plane", fontsize=titlefontsize)
    fig.tight_layout()
    plt.savefig(os.path.join(out_path, f"{region}-{metric}.jpg"), dpi=1000, bbox_inches='tight')
    plt.close(fig=fig);

In [10]:
titlefontsize = 32
fontsize = 24

monkey = "both"
selectivity = "fast"
out_path = f"G:/Codes/Processing/out/svm-tt/{monkey}-{selectivity}"
metric = "cfn"

for region in ["it", "pfc"]:
    fig, axs = plt.subplots(1, 2, figsize=np.array([30, 13]), dpi=30)#/2.54)
    time = np.load(os.path.join(out_path, f"{region.lower()}-super-ordinate-tim.npy"))
    for ax, ordination in zip(axs, ["super-ordinate", "mid-level"]):
        m = np.load(os.path.join(out_path, f"{region.lower()}-{ordination}-{metric}.npy"))
        m = np.trace(m, axis1=0, axis2=1) / m.sum((0, 1))
        h = ax.contourf(time, time, m.mean(2), 400, cmap='seismic')
        cbar = fig.colorbar(h, ax=ax, shrink=.6, format=ticker.FuncFormatter(myfmt))
        cbar.ax.tick_params(labelsize=fontsize)
        ax.set_title(ordination, fontsize=titlefontsize)

        ax.axis("image")
        ax.set_xlabel('train time (ms)', fontsize=fontsize)
        ax.tick_params(axis='both', which='major', labelsize=fontsize)    

    axs[0].set_ylabel('test time (ms)', fontsize=fontsize)

    fig.suptitle(f"{region.upper()}: Decoder Accuracy", fontsize=titlefontsize)
    fig.tight_layout()
    plt.savefig(os.path.join(out_path, f"{region}-{metric}.jpg"), dpi=1000, bbox_inches='tight')
    plt.close(fig=fig);